In [15]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.user_credential import UserCredential
from io import BytesIO
import pyodbc
import pandas as pd
import re
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
import pandas as pd
from io import StringIO
from office365.runtime.auth.user_credential import UserCredential
import smtplib
from email.message import EmailMessage
import os
from datetime import datetime, timedelta


In [16]:
# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites"
username = "emailsharepoint"
password = "emailpassword"

# File URLs for MWH and PWH
file_urls = [
    "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch/merged_data.xlsx"
]

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Initialize empty DataFrames for MWH and PWH
merged_data = None

# Read each file and convert to pandas DataFrame
for file_url in file_urls:
    response = File.open_binary(ctx, file_url)
    excel_data = BytesIO(response.content)
    
    # Read all sheets into a dictionary of DataFrames
    merged_data = pd.read_excel(excel_data, sheet_name=None)

# Check if the file was loaded correctly and print the available sheet names
if merged_data is not None:
    print("Available sheets in merged_data Replenishment:")
    print(merged_data.keys())  # List all sheet names in MWH
else:
    print("Error: merged_data file could not be loaded or is empty.")

merged_data_name = 'Merged_Data'  # Adjust this if necessary
if merged_data_name  in merged_data.keys():
    merged_data = merged_data[merged_data_name ]
    
    # Display the first few rows of the sheet
    print("merged_data Replenishment Sheet Data:")
    print(merged_data.head())
else:
    print(f"Sheet '{merged_data_name}' not found in merged_data_name.")

Available sheets in merged_data Replenishment:
dict_keys(['Merged_Data'])
merged_data Replenishment Sheet Data:
   PRODUCT_CODE  KOTTE_STOCK  COLOMBO_STOCK  ATTIDIYA_STOCK  PWH_STOCK  \
0           1.0         16.0           20.0             9.0        0.0   
1           2.0         18.0           10.0            26.0        0.0   
2           3.0          9.0           21.0            14.0        0.0   
3           7.0         27.0           13.0            18.0        0.0   
4          10.0          2.0            0.0             2.0        0.0   

   MWH_STOCK  Items Code                                   Item Description  \
0        6.0           1  ADULT CHICKEN VEGETABLE 3KG - PEDIGREE - SKU 0...   
1       46.0           2       ADULT OCEAN FISH 480G - WHISKAS - SKU 000002   
2       36.0           3             ADULT TUNA 480G - WHISKAS - SKU 000003   
3        0.0           7  ANLENE LOW FAT MILK POWDER 400G - ANCHOR - SKU...   
4        0.0          10  CELESTE X-MAS APPLE CI

In [17]:
# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites"
username = "emailsharepoint"
password = "emailpassword"

# File URLs for MWH and PWH
file_urls = [
    "/sites/Stock/Shared Documents/Transfer-Outs Request/MWH Replenishment.xlsx"
]

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Initialize empty DataFrames for MWH and PWH
df_Pwh = None

# Read each file and convert to pandas DataFrame
for file_url in file_urls:
    response = File.open_binary(ctx, file_url)
    excel_data = BytesIO(response.content)
    
    # Read all sheets into a dictionary of DataFrames
    df_Pwh = pd.read_excel(excel_data, sheet_name=None)

# Check if the file was loaded correctly and print the available sheet names
if df_Pwh is not None:
    print("Available sheets in MWH Replenishment:")
    print(df_Pwh.keys())  # List all sheet names in MWH
else:
    print("Error: MWH file could not be loaded or is empty.")

# Assuming there's a sheet called 'MWH', if it exists, we access it
Pwh_sheet_name = 'MWH'  # Adjust this if necessary
if Pwh_sheet_name in df_Pwh.keys():
    Pwh_sheet1_df = df_Pwh[Pwh_sheet_name]
    
    # Display the first few rows of the sheet
    print("MWH Replenishment Sheet Data:")
   # print(Pwh_sheet1_df.head())
else:
    print(f"Sheet '{Pwh_sheet_name}' not found in PWH Replenishment.")

Available sheets in MWH Replenishment:
dict_keys(['Summary', 'MWH'])
MWH Replenishment Sheet Data:


In [18]:
print(Pwh_sheet1_df.columns)


Index(['ITEM CODE', 'ITEM DESCRPTION', 'SUPPLIER', 'DEPARTMENT',
       'SUB DEPARTMENT', 'WAREHOUSE', 'BARCODE', 'SELLING PRICE',
       'SKU Category', 'Priority', 'Col 6', 'Att', 'Ekotte', 'WAREHOUSE SIH',
       'Final Replenishment Qty Col', 'Final Replenishment Qty Att',
       'Final Replenishment Qty Ekotte', 'Total', 'Balance', 'Case Config'],
      dtype='object')


In [19]:
# Drop specified columns from the DataFrame
Pwh_sheet1_df.drop(columns=['SKU Category', 'SUPPLIER', 'DEPARTMENT', 'SUB DEPARTMENT',  'Priority', 'Col 6', 'Att', 'Ekotte','Balance','Case Qty'], inplace=True, errors='ignore')
Pwh_sheet1_df

,ITEM CODE,ITEM DESCRPTION,WAREHOUSE,BARCODE,SELLING PRICE,WAREHOUSE SIH,Final Replenishment Qty Col,Final Replenishment Qty Att,Final Replenishment Qty Ekotte,Total,Case Config
0,4251,APPIFIZZ GREEN 250ML - SUN CRUSH - SKU 004251,MWH,4796027480074,150,192,24,0,0,24,24
1,3432,APPLE JUICE 180ML - MY JUICEE - SKU 003432,MWH,4792116204132,120,72,0,24,0,24,24
2,3433,APPLE JUICE 1L - MY JUICEE - SKU 003433,MWH,4792116204033,600,72,0,0,12,12,12
3,2471,AQUAFRESH 1L CLASSIC - SKU 002471,MWH,4796000210032,155,504,0,0,36,36,12
4,2473,AQUAFRESH 5L - SKU 002473,MWH,4796000210056,485,96,52,0,0,52,4
...,...,...,...,...,...,...,...,...,...,...,...
251,1910,LIFE STYLE ROUGH RIDER 13G - SKU 001910,MWH,8850956032141,200,18,6,0,0,6,12
252,2463,POSTINOR 1 - SKU 002463,MWH,5997001360484,400,120,24,24,0,48,24
253,2462,POSTINOR 2 - SKU 002462,MWH,5997001360170,295,60,30,0,30,60,30
254,1913,SKYN NON - LATEX CONDOMS 13.3G - SKU 001913,MWH,8850956032240,680,48,0,12,12,24,12


In [20]:
import pandas as pd


# Perform the merge
merged_df = pd.merge(merged_data, Pwh_sheet1_df, left_on='PRODUCT_CODE', right_on='ITEM CODE', how='left')
# Replace NaN values with 0
merged_df = merged_df.fillna(0)
# Display merged DataFrame
merged_df['SKU Category'] = merged_df['SKU Category'].str.upper()

In [21]:
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import pandas as pd
from io import BytesIO

# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites"
username = "emailsharepoint"
password = "emailpassword"

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Example: Create a DataFrame `merge_df`
# Replace this with your actual DataFrame creation or loading logic


# Create a BytesIO buffer to hold the CSV data
csv_buffer = BytesIO()


# Save the merged DataFrame as a CSV in the buffer
merged_df.to_csv(csv_buffer, index=False)  # Set index=True if you want to include the index

# Reset buffer position to the beginning
csv_buffer.seek(0)

# Specify the target folder and file name for the CSV upload
target_folder_url = "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch"
target_file_name = "MWH_merged_data_after.csv"  # Name of the CSV file to be uploaded
target_folder = ctx.web.get_folder_by_server_relative_url(target_folder_url)

# Check if the file already exists and delete it
try:
    existing_file = target_folder.files.get_by_url(target_file_name)
    existing_file.delete_object().execute_query()
    print(f"Existing file '{target_file_name}' deleted.")
except Exception as e:
    print(f"No existing file to delete: {e}")

# Upload the new CSV file
target_file = target_folder.upload_file(target_file_name, csv_buffer).execute_query()

# Print the server-relative URL of the uploaded file
print(f"File '{target_file.serverRelativeUrl}' has been uploaded.")


Existing file 'MWH_merged_data_after.csv' deleted.
File '/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch/MWH_merged_data_after.csv' has been uploaded.


In [22]:
print(merged_df.columns)

Index(['PRODUCT_CODE', 'KOTTE_STOCK', 'COLOMBO_STOCK', 'ATTIDIYA_STOCK',
       'PWH_STOCK', 'MWH_STOCK', 'Items Code', 'Item Description', 'Supplier',
       'Department', 'Warehouse', 'SKU Category', 'Col Active / Inactive ',
       'Att Active / Inactive ', 'Kotte Active / Inactive ', 'ITEM CODE',
       'ITEM DESCRPTION', 'WAREHOUSE', 'BARCODE', 'SELLING PRICE',
       'WAREHOUSE SIH', 'Final Replenishment Qty Col',
       'Final Replenishment Qty Att', 'Final Replenishment Qty Ekotte',
       'Total', 'Case Config'],
      dtype='object')


In [23]:
merged_df[['KOTTE_STOCK', 'COLOMBO_STOCK', 'ATTIDIYA_STOCK']] = merged_df[['KOTTE_STOCK', 'COLOMBO_STOCK', 'ATTIDIYA_STOCK']].clip(lower=0)


In [24]:
import pandas as pd
from io import BytesIO
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential

# SharePoint site and authentication details
site_url = "https://lkceleste.sharepoint.com/sites"
username = "emailsharepoint"
password = "emailpassword"

# Authenticate SharePoint context
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Folder path where the CSV files are stored
target_folder_url = "/sites/Stock/Shared Documents/Transfer-Outs Request/dont_touch"

# Function to read a CSV file from SharePoint into a Pandas DataFrame
def read_csv_from_sharepoint(file_name, folder_url, ctx):
    file_url = f"{folder_url}/{file_name}"  # Full relative URL of the file

    # Create a memory buffer to store the downloaded file
    buffer = BytesIO()
    
    # Get the file content from SharePoint
    file = ctx.web.get_file_by_server_relative_url(file_url)
    file.download(buffer).execute_query()  # Download file into buffer

    # Move the buffer's pointer to the beginning
    buffer.seek(0)

    # Read the buffer content into a Pandas DataFrame
    df = pd.read_csv(buffer)
    return df

# Read the uploaded CSV files
pivot_table_1_df = read_csv_from_sharepoint("MWH_active_counts.csv", target_folder_url, ctx)
pivot_table_2_df = read_csv_from_sharepoint("MWH_out_of_stock_counts.csv", target_folder_url, ctx)

# Display the loaded DataFrames
print("BEFORE MWH Active Counts Data:")
print(pivot_table_1_df)

print("\nBEFORE MWH Out-of-Stock Counts and Percentages Data:")
print(pivot_table_2_df)


BEFORE MWH Active Counts Data:
  SKU Category  COLOMBO  ATTIDIYA  KOTTE
0            A      673       667    673
1            B      569       547    569
2            C      931       779    905
3           NL      132       118    128

BEFORE MWH Out-of-Stock Counts and Percentages Data:
  SKU Category  COLOMBO  ATTIDIYA  KOTTE  COLOMBO Percentage  \
0            A      103       101    100           15.304606   
1            B      145       119    133           25.483304   
2            C      254       251    284           27.282492   
3           NL       57        62     62           43.181818   

   ATTIDIYA Percentage  KOTTE Percentage  
0            15.142429         14.858841  
1            21.755027         23.374341  
2            32.220796         31.381215  
3            52.542373         48.437500  


In [25]:
import pandas as pd

# Filter DataFrame for Warehouse 'PWH'
filtered_df = merged_df[merged_df['Warehouse'] == 'MWH']

# PIVOT 1: Active Product Counts

def get_active_counts(df, active_col):
    return df[df[active_col].str.strip() == 'A'].groupby('SKU Category')['PRODUCT_CODE'].count()

colombo_counts = get_active_counts(filtered_df, 'Col Active / Inactive ')
attidiya_counts = get_active_counts(filtered_df,  'Att Active / Inactive ')
kotte_counts = get_active_counts(filtered_df,  'Kotte Active / Inactive ')

pivot_table_1 = pd.DataFrame({
    'COLOMBO': colombo_counts,
    'ATTIDIYA': attidiya_counts,
    'KOTTE': kotte_counts
}).fillna(0).astype(int)

print("\nAFTER Count of Active Products for MWH by SKU Category and Stock Location")
print(pivot_table_1)

# PIVOT 2: Out-of-Stock Counts and Percentages

def get_out_of_stock_counts(df, stock_col, active_col, update):
    return df[(df[active_col].str.strip() == 'A') & ((df[stock_col] + df[update]) <= 0)].groupby('SKU Category')['PRODUCT_CODE'].count()

colombo_oos = get_out_of_stock_counts(filtered_df, 'COLOMBO_STOCK', 'Col Active / Inactive ','Final Replenishment Qty Col')
attidiya_oos = get_out_of_stock_counts(filtered_df, 'ATTIDIYA_STOCK', 'Att Active / Inactive ','Final Replenishment Qty Att')
kotte_oos = get_out_of_stock_counts(filtered_df, 'KOTTE_STOCK', 'Kotte Active / Inactive ','Final Replenishment Qty Ekotte')

pivot_table_oos = pd.DataFrame({
    'COLOMBO': colombo_oos,
    'ATTIDIYA': attidiya_oos,
    'KOTTE': kotte_oos
}).fillna(0).astype(int)

oos_percentage = (pivot_table_oos / pivot_table_1.replace(0, float('nan'))) * 100
final_pivot_table_2 = pivot_table_oos.copy()
final_pivot_table_2['COLOMBO Percentage'] = oos_percentage['COLOMBO']
final_pivot_table_2['ATTIDIYA Percentage'] = oos_percentage['ATTIDIYA']
final_pivot_table_2['KOTTE Percentage'] = oos_percentage['KOTTE']

print("\nAFTER Count of Out-of-Stock Products and Percentages for MWH by SKU Category and Stock Location")
print(final_pivot_table_2)

# PIVOT 3: Out-of-Stock Items with Sendset Available

def get_active_count(df, stock_col, active_col, sendset,warehouse):
    return df[(df[active_col].str.strip() == 'A') & (df[stock_col] <= 0)  & (df[warehouse] > 0)].groupby('SKU Category')['PRODUCT_CODE'].count()

colombo_count = get_active_count(filtered_df, 'COLOMBO_STOCK', 'Col Active / Inactive ', 'Final Replenishment Qty Col','MWH_STOCK')
attidiya_count = get_active_count(filtered_df, 'ATTIDIYA_STOCK', 'Att Active / Inactive ', 'Final Replenishment Qty Att','MWH_STOCK')
kotte_count = get_active_count(filtered_df, 'KOTTE_STOCK', 'Kotte Active / Inactive ', 'Final Replenishment Qty Ekotte','MWH_STOCK')

pivot_table_3 = pd.DataFrame({
    'COLOMBO': colombo_count,
    'ATTIDIYA': attidiya_count,
    'KOTTE': kotte_count
}).fillna(0).astype(int)

oos_percentage_3 = (pivot_table_3 / pivot_table_1.replace(0, float('nan'))) * 100
final_pivot_table_3 = pivot_table_3.copy()
final_pivot_table_3['COLOMBO Percentage'] = oos_percentage_3['COLOMBO']
final_pivot_table_3['ATTIDIYA Percentage'] = oos_percentage_3['ATTIDIYA']
final_pivot_table_3['KOTTE Percentage'] = oos_percentage_3['KOTTE']

print("\nAFTER Count of Out-of-Stock Products with Sendset Available and Percentages for MWH by SKU Category and Stock Location")
print(final_pivot_table_3)

# === PIVOT 4: DIFFERENCE BETWEEN BEFORE AND AFTER ===
pivot_table_difference = pivot_table_2_df.set_index('SKU Category') - final_pivot_table_2
pivot_table_difference.columns = [
    'COLOMBO Diff', 'ATTIDIYA Diff', 'KOTTE Diff',
    'COLOMBO Percentage Diff', 'ATTIDIYA Percentage Diff', 'KOTTE Percentage Diff'
]



AFTER Count of Active Products for MWH by SKU Category and Stock Location
              COLOMBO  ATTIDIYA  KOTTE
SKU Category                          
A                 673       667    673
B                 569       547    569
C                 931       779    905
NL                132       118    128

AFTER Count of Out-of-Stock Products and Percentages for MWH by SKU Category and Stock Location
              COLOMBO  ATTIDIYA  KOTTE  COLOMBO Percentage  \
SKU Category                                                 
A                 103       100    100           15.304606   
B                 143       118    130           25.131810   
C                 251       247    280           26.960258   
NL                 57        62     62           43.181818   

              ATTIDIYA Percentage  KOTTE Percentage  
SKU Category                                         
A                       14.992504         14.858841  
B                       21.572212         22.847100  
C    

In [26]:
!pip install schedule

In [27]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [28]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email Configuration
SMTP_SERVER = "smtp.office365.com"
SMTP_PORT = 587
EMAIL_ADDRESS = "senderemail"
EMAIL_PASSWORD = "password"


cc_emails = ["nimesh.lk","darshana.lk"]
# Convert pivot tables to HTML format with specific names
to_email = ["pradeepa.lk","Nadeeshan.lk"]

tables_html = f"""
<html>
<body>
    <h2>📌 BEFORE MWH Active Counts Data</h2>
    {pivot_table_1_df.to_html(index=False)}

    <h2>📌 AFTER Count of Active Products for MWH by SKU Category and Stock Location</h2>
    {pivot_table_1.to_html(index=True)}


    <h2>📌 BEFORE MWH Out-of-Stock Counts and Percentages Data</h2>
    {pivot_table_2_df.to_html(index=False)}


    <h2>📌 AFTER Count of Out-of-Stock Products and Percentages for MWH by SKU Category and Stock Location</h2>
    {final_pivot_table_2.to_html(index=True)}



    <h2>📌 AFTER REQUEST Count of Out-of-Stock Products with Sendset Available and Percentages for MWH by SKU Category and Stock Location</h2>
    {pivot_table_difference.to_html(index=True)}
</body>
</html>
"""
#    <h2>📌 AFTER REQUEST Count of Out-of-Stock Products with Sendset Available and Percentages for MWH by SKU Category and Stock Location</h2>
#    {final_pivot_table_3.to_html(index=True)}


# Create email message
msg = MIMEMultipart()
msg['From'] = EMAIL_ADDRESS
msg['To'] = ", ".join(to_email)  # Convert list to comma-separated string
msg['Cc'] = ", ".join(cc_emails)  # Convert list to comma-separated string
msg['Subject'] = "please check and signoff📊 MWH Stock Report - 9:00 AM"
msg.attach(MIMEText(tables_html, 'html'))

# Send Email
try:
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        recipients = to_email + cc_emails  # Merge lists correctly
        server.sendmail(EMAIL_ADDRESS, recipients, msg.as_string())

    print("✅ Email sent successfully!")

except Exception as e:
    print(f"❌ Failed to send email: {e}")


✅ Email sent successfully!
